# Training ML model using TFX

Once we have out transformed data in place, we can move on to the Model training part. Model building is fairly simple in most cases compared actual productionizing the product and therefore tensorflow provides facilities to train models through a pipeline with hyperparameter tuning support. 

To test this functionality we need a model architecture already defined and I am using the model architecture as outlined in the reference book without any change.

Also one major point is that, we need to gurantee the feature input names for the models are exactly same as preprocessing_fn in the pipeline.

In [1]:
import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import os
from tfx.components import CsvExampleGen
import tensorflow as tf
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import Transform
from tfx.components import ExampleValidator

context = InteractiveContext(pipeline_root='data/tfx')

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, "data", 'CSVs')

example_gen = CsvExampleGen(data_dir)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [3]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [4]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                        infer_feature_shape=True)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [5]:
example_validator = ExampleValidator(
            statistics=statistics_gen.outputs['statistics'],
            schema=schema_gen.outputs['schema'])

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

transformer = Transform(
                    examples=example_gen.outputs['examples'],
                    schema=schema_gen.outputs['schema'],
                    module_file=os.path.join(os.getcwd(), 'scripts/tft_transform_pipeline.py')
                )

context.run(transformer)

Instructions for updating:
Use ref() instead.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Once the data pipeline in place, we can define our model as we need. But it's inputs should only use the ones outlined by the transformer step.

In [7]:
%%writefile scripts/tft_trainer_support.py

import tensorflow as tf
import tensorflow_hub as hub

LABEL_KEY = "consumer_disputed"
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}
BUCKET_FEATURES = {"zip_code": 10}
TEXT_FEATURES = {"consumer_complaint_narrative": None}

def tranformed_name(key):
    return key + "_xf"

def get_model():

    input_features = []
    
    # These 2 types would go to the broad network segment!
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim+1,), name=tranformed_name(key))
        )

    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim+1,), name=tranformed_name(key))
        )

    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,), name=tranformed_name(key), dtype=tf.string)
        )

    inputs = input_features + input_texts

    # URL to get the sentence embedding model from tensorflow hub
    MODULE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed_layer = hub.KerasLayer(MODULE_URL)
    reshaped_text_input = tf.reshape(input_texts[0], [-1])
    embedded_text_inputs = embed_layer(reshaped_text_input)
    reshaped_embeddings = tf.keras.layers.Reshape((512,), input_shape=(1, 512))(embedded_text_inputs)

    deep = tf.keras.layers.Dense(256, activation='relu')(reshaped_embeddings)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)
    
    both = tf.keras.layers.concatenate([deep, wide])
    output = tf.keras.layers.Dense(1, activation='sigmoid')(both)

    keras_model = tf.keras.models.Model(inputs, output) 
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        loss='binary_crossentropy',
                        metrics=[
                            tf.keras.metrics.BinaryAccuracy(),
                            tf.keras.metrics.TruePositives()
                        ])
    return keras_model


Overwriting scripts/tft_trainer_support.py


This is not a typical architecture I have seen tbh! So really interesting. Idea is connecting a deep and wide networks together to acheive a single task with different types of features. More about this atchitecture can be found in [this google blog post](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html). 

Now once the model is done, we can move on to integrating it with other pipeline components.

### TFX Trainer component

This component handles the training part of out pipeline. Compared to normal tensorflow model development, this component require additional coding effort. But this helps us to standardize data transformation and validation parts from the pipeline and deploying the model.

Trainer component require few components as inputs to function.

1. data scheme generated by the data validation step.
2. transformed data and transformation graph used by it (transform_fn)
3. training parameters
4. a file containing a run_fn which define the training process.

The Trainer component look for a run_fn() in the provided module and use it as the entry point for the training process. Depending on whether we are running in interactive environment or in production may need to change the way we provide the module with run_fn().

Below is an example run_fn() for the model we build earlier. (Note this is not keras specific.)

it does the loading of training and validation data, defining the model and compiling, training the model and exporting the model to be evaluated in the next pipeline step.

In [8]:
%%writefile scripts/tft_trainer_script.py

from scripts.tft_trainer_support import *
import tensorflow_transform as tft

def _gzip_reader_fn(filenames):
    '''
    Helper to read tfrecords compressed using gzip compression
    '''
    return tf.data.TFRecordDataset(filenames,
        compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, batch_size=32):
    '''
    Helper function to return dataset as batches.
    '''
    transformed_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=tranformed_name(LABEL_KEY))
    return dataset

def _get_serve_prediction_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    # Loading the transform graph from the Transform component output
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        # Parsing the request data to tf.Example format
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # Do the transformation based on the loaded transformation graph.
        transformed_features = model.tft_layer(parsed_features)

        # Do the inference
        outputs = model(transformed_features)

        # Return output
        return {"outputs": outputs}

    return serve_tf_examples_fn

def run_fn(args):

    # Loading data to the training process
    tf_transform_output = tft.TFTransformOutput(args.transform_output)
    train_dataset = input_fn(args.train_files, tf_transform_output) 
    eval_dataset = input_fn(args.eval_files, tf_transform_output)

    model = get_model()

    model.fit(
        train_dataset,
        steps_per_epoch=args.train_steps,
        validation_data=eval_dataset,
        validation_steps=args.eval_steps
    )

    signatures = {
        'serving_default':
            _get_serve_prediction_fn(model, tf_transform_output)\
                    .get_concrete_function(
                        tf.TensorSpec(
                            shape=[None],
                            dtype=tf.string,
                            name='examples')
                    )
    }

    model.save(args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting scripts/tft_trainer_script.py


In the above input_fn is used to load the compressed, preprocessed datasets that were generated by the previous Transform step. To do this, we need to pass the tf_transform_output parameter which gives the data schema to load data from TFRecord data structures. Then it returns a generator object which can supply one batch at a time to the model.

Then we have the model defining and training part. Only change in here compared to normal flow is we are using num_of_steps rather than num_of_epochs.

In the last part of the run_fn we work on exporting the model using the tensorflow's SavedModel format. There we have defined a signature object based on the graph generated by the _get_serve_prediction_fn. This function get exported by the run_fn as part of the model signature. And when we deploy the model, every request we send to the model will get serialized into tf.Example records and will get processed before passed on to the model prediction. Very cool!


Once we have all above in place, we can turn into running the trainer component in the pipeline. It requires following inputs.

- python file with run_fn()
- transformed examples generated by the Transform component
- transform graph generated by the Transform component
- schema generated by the data validation component
- number of training and validation steps

Below is the definition of the trainer component with all the above.

In [9]:
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2


TRAINING_STEPS = 1000
EVALUATION_STEPS = 100


trainer = Trainer(
    module_file='scripts/tft_trainer_script.py',
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor), # Here we overload the executor to use the run_fn()
    transformed_examples=transformer.outputs['transformed_examples'],
    transform_graph=transformer.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS)
)

Please refer the documentation for more details! This is just a outline for a basic implementation.

In [10]:
context.run(trainer)

1000/1000 [==============================] - 32s 29ms/step - loss: 0.5251 - binary_accuracy: 0.7650 - true_positives: 25.0000 - val_loss: 0.5103 - val_binary_accuracy: 0.7691 - val_true_positives: 8.0000


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

After this step, model is exported and will get registered in the metadata store. The next components of the pipeline can pickup the model from there.

Important point to note is that, component next inline of tfx, expect the model to be saved in SavedModel format which include the computational graph along with the transformation graph (preprocessing steps).

This trainer's executor can be override to our need. For example we can use google cloud AI platform for model training instead of using local pipeline. More on these can be found on the documentation.